# Save frames

In [6]:
import cv2
import os
from datetime import datetime

# === Settings ===
output_dir = "captures"
camera_index = 0  # Change if needed
chessboard_size = (5,8)  # Number of inner corners (columns, rows) - adjust if needed

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open webcam
cap = cv2.VideoCapture(camera_index)
if not cap.isOpened():
    print(f"Error: Could not open camera index {camera_index}")
    exit()

print("Press SPACE to capture image, ESC to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break

    # Convert to grayscale for chessboard detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = 255-gray

    # Try to find chessboard
    ret_corners, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    display_frame = frame.copy()

    if ret_corners:
        # Optional: Refine corners for better visualization
        criteria = (cv2.TermCriteria_EPS + cv2.TermCriteria_MAX_ITER, 30, 0.001)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        cv2.drawChessboardCorners(display_frame, chessboard_size, corners2, ret_corners)
        cv2.putText(display_frame, "Chessboard Detected!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show live preview
    cv2.imshow("Camera Capture + Chessboard Detection", display_frame)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC
        break
    elif key == 32:  # SPACE
        # Save original frame (not the one with drawn corners)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = os.path.join(output_dir, f"capture_{timestamp}.png")
        cv2.imwrite(filename, frame)
        print(f"Saved: {filename}")

cap.release()
cv2.destroyAllWindows()


Press SPACE to capture image, ESC to exit.


QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to tar

Saved: captures/capture_20250616_184914_642460.png
Saved: captures/capture_20250616_184917_261276.png
Saved: captures/capture_20250616_184921_391728.png
Saved: captures/capture_20250616_184924_019891.png
Saved: captures/capture_20250616_184925_791898.png
Saved: captures/capture_20250616_184929_205719.png
Saved: captures/capture_20250616_184930_794414.png
Saved: captures/capture_20250616_184932_603319.png
Saved: captures/capture_20250616_184934_962772.png
Saved: captures/capture_20250616_184938_079485.png
Saved: captures/capture_20250616_184941_315577.png


# Estimate Params

In [7]:
import cv2
import numpy as np
import os
import glob

# === Settings ===
images_folder = "captures"
chessboard_size = (5,8)  # Number of inner corners per chessboard row and column (change if needed)
square_size = 0.02  # Real-world size of a square (set to 1.0 if you only want relative calibration)

# Prepare object points like (0,0,0), (1,0,0), (2,0,0), ..., assuming square_size units
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
objp *= square_size

# Arrays to store points from all images
objpoints = []  # 3D points in real world space
imgpoints = []  # 2D points in image plane

# Get all images from folder
image_paths = glob.glob(os.path.join(images_folder, "*.png"))

if len(image_paths) == 0:
    print(f"No images found in {images_folder}")
    exit()

for path in image_paths:
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = 255-gray

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        # Refine corner locations for better accuracy
        criteria = (cv2.TermCriteria_EPS + cv2.TermCriteria_MAX_ITER, 30, 0.001)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Optional: Draw and show
        cv2.drawChessboardCorners(img, chessboard_size, corners2, ret)
        cv2.imshow('Detected Chessboard', img)
        cv2.waitKey(300)  # Show for 300 ms
    else:
        print(f"Chessboard not found in: {path}")

cv2.destroyAllWindows()

# Calibration
if len(objpoints) < 5:
    print("Not enough valid chessboard images for calibration (need at least 5)")
    exit()

ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

# Results
print("\n=== Calibration Results ===")
print("Camera Matrix (Intrinsic Parameters):\n", camera_matrix)
print("\nDistortion Coefficients:\n", dist_coeffs.ravel())
print("\nReprojection Error (RMS):", ret)

# Save results
np.savez('camera_calibration.npz',
         camera_matrix=camera_matrix,
         dist_coeffs=dist_coeffs,
         rvecs=rvecs,
         tvecs=tvecs)

print("\nCalibration saved to 'camera_calibration.npz'")


QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to tar


=== Calibration Results ===
Camera Matrix (Intrinsic Parameters):
 [[608.04586403   0.         334.57525786]
 [  0.         607.64978458 247.30819715]
 [  0.           0.           1.        ]]

Distortion Coefficients:
 [-2.52199939e-01  1.05441015e+00  3.92627518e-03  1.23518714e-03
 -1.59275218e+00]

Reprojection Error (RMS): 0.0982775810780699

Calibration saved to 'camera_calibration.npz'


# test undistortion

In [8]:
import cv2
import numpy as np

# === Load Calibration ===
calib_file = 'camera_calibration.npz'
data = np.load(calib_file)
camera_matrix = data['camera_matrix']
dist_coeffs = data['dist_coeffs']

print("Loaded camera matrix:\n", camera_matrix)
print("\nLoaded distortion coefficients:\n", dist_coeffs.ravel())

# === Choose input source ===
use_webcam = True  # Set to False if you want to undistort a saved image

if use_webcam:
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Cannot open webcam")
        exit()

    print("Press ESC to exit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Undistort
        undistorted = cv2.undistort(frame, camera_matrix, dist_coeffs, None, camera_matrix)

        # Show side by side
        combined = np.hstack((frame, undistorted))
        cv2.imshow('Original (left) vs Undistorted (right)', combined)

        key = cv2.waitKey(1)
        if key == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()

else:
    # === Test on saved image ===
    img = cv2.imread('captures/your_image.png')  # Change path
    if img is None:
        print("Error: Could not load image.")
        exit()

    undistorted = cv2.undistort(img, camera_matrix, dist_coeffs, None, camera_matrix)

    # Show side by side
    combined = np.hstack((img, undistorted))
    cv2.imshow('Original (left) vs Undistorted (right)', combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Loaded camera matrix:
 [[608.04586403   0.         334.57525786]
 [  0.         607.64978458 247.30819715]
 [  0.           0.           1.        ]]

Loaded distortion coefficients:
 [-2.52199939e-01  1.05441015e+00  3.92627518e-03  1.23518714e-03
 -1.59275218e+00]
Press ESC to exit.


QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to target thread (0x5ee1acd0e2c0)

QObject::moveToThread: Current thread (0x5ee1acd0e2c0) is not the object's thread (0x5ee1ad26fd50).
Cannot move to tar